# Quality Control

In [ ]:
# import necessary modules
import glob, os
import pandas as pd
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import numpy as np
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from scipy.optimize import curve_fit
import json

#------------------------------------------
colours=["#679436","#ffd400","#d7907b","#6c4b5e","#87c38f","#c6d8ff","#9b1d20","#002626","#337ca0","#9d8df1","#679436","#ffd400","#d7907b","#6c4b5e","#87c38f","#c6d8ff","#9b1d20","#002626","#337ca0","#9d8df1","#679436","#ffd400","#d7907b","#6c4b5e","#87c38f","#c6d8ff","#9b1d20","#679436","#ffd400","#d7907b","#6c4b5e","#87c38f","#c6d8ff","#9b1d20","#002626","#337ca0","#9d8df1","#679436","#ffd400","#d7907b","#6c4b5e","#87c38f","#c6d8ff","#9b1d20","#002626","#337ca0","#9d8df1","#679436","#ffd400","#d7907b","#6c4b5e","#87c38f","#c6d8ff","#9b1d20",]


# import necessary modules
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import glob, os
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_style("ticks",{'axes.grid' : True})
plt.rcParams["axes.linewidth"] = 1.5
plt.rcParams["xtick.major.width"] = 1.5
plt.rcParams["ytick.major.width"] = 1.5
plt.rcParams["xtick.major.size"] = 8
plt.rcParams["ytick.major.size"] = 8
plt.rcParams["axes.titlepad"] = 20

plt.rcParams['svg.fonttype'] = 'none'
plt.rcParams["axes.titlesize"] = 30
plt.rcParams['axes.labelsize'] = 23.5
plt.rcParams['xtick.labelsize'] = 18
plt.rcParams['ytick.labelsize'] = 18
plt.rcParams['legend.fontsize'] = 18
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Liberation Sans']
plt.rcParams['text.usetex'] = False

plt.rcParams['svg.fonttype'] = 'none'

In [ ]:
clean_dir=snakemake.params.clean_dir
raw_dir=snakemake.params.raw_dir
qc_dir=snakemake.params.qc_dir

SAMPLES=list(snakemake.params.samples)
forward_tag="_" + snakemake.params.forward_tag
reverse_tag="_" + snakemake.params.reverse_tag

input_histograms=list(snakemake.input.histograms)
illumina_preqc=snakemake.input.preqc_txt
illumina_postqc=snakemake.input.postqc_txt
#------------------------------------------
output_kmer_png=snakemake.output.kmer_png
output_kmer_svg=snakemake.output.kmer_svg
output_kmer_png_fitted=snakemake.output.kmer_fit_png
output_kmer_svg_fitted=snakemake.output.kmer_fit_svg
output_kmer_fit_html=snakemake.output.kmer_fit_html
output_qc_summary_html=snakemake.output.qc_summary_html
output_percentage_kept_reads_png=snakemake.output.percentage_kept_reads_png
output_percentage_kept_reads_svg=snakemake.output.percentage_kept_reads_svg
output_percentage_kept_Mbp_png=snakemake.output.percentage_kept_Mbp_png
output_percentage_kept_Mbp_svg=snakemake.output.percentage_kept_Mbp_svg
output_step_qc_reads_html=snakemake.output.step_qc_reads_html
output_steps_qc_reads_png=snakemake.output.steps_qc_reads_png
output_steps_qc_reads_svg=snakemake.output.steps_qc_reads_svg
output_steps_qc_percentage_png=snakemake.output.steps_qc_percentage_png
output_steps_qc_percentage_svg=snakemake.output.steps_qc_percentage_svg
output_supperdedupper_html=snakemake.output.supperdedupper_html
output_supperdedupper_png=snakemake.output.supperdedupper_png
output_supperdedupper_svg=snakemake.output.supperdedupper_svg

## KMER rarefraction

In [ ]:
# set figure size and font scale for seaborn
plt.figure(figsize=(12,12))

# Sort input_histograms and print the sorted list
input_histograms.sort()

# Sort SAMPLES
SAMPLES.sort()

# Initialize n and read_max as 0
n=0
read_max=0

# Loop through each histogram in input_histograms
for h in input_histograms:
    # Read the histogram file as a pandas dataframe
    df=pd.read_csv(h, sep="\t")
    
    # Rename columns and select only the "counts" and "percent" columns
    df.columns=["counts", "percent", "c", "d", "e", "f", "g", "h", "rand_cnt", "j"]
    df=df[["counts", "percent"]]
    
    # Convert percent to percentage of missing kmers and counts to millions of reads
    df["percent"]=(100-df["percent"])
    df["counts"]=df["counts"]/1000000
    
    # Plot the data as a line plot
    ax = sns.lineplot(x="counts", y="percent", data=df,err_style='band',color=colours[n], label=h.split("/")[-1].split("_kmer")[0], linewidth=0.5)
    
    # Add a vertical line at the x-value corresponding to the maximum count and update read_max if needed
    plt.axvline(df["counts"].max(), 0,1, linestyle="--", color=colours[n], alpha=0.3)
    read_max=max(read_max,df["counts"].max())
    
    # Increment n
    n=n+1
    
    # Set y- and x-axis limits
    ax.set(ylim=(0,110))
    ax.set(xlim=(0, read_max*1.2))

    # Set x- and y-axis labels and legend
    ax.set_xlabel("Read count (Million reads)")
    ax.set_ylabel("Known k-mers (%)")
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    
    # Save the figure as png and svg files
    ax.figure.savefig(output_kmer_png, format="png", bbox_inches = "tight",transparent=True)
    ax.figure.savefig(output_kmer_svg, format="svg", bbox_inches = "tight",transparent=True)
    
# Show the plot
plt.show()


# KMER rarefraction log fit

In [ ]:
# set figure size and font scale for seaborn
plt.figure(figsize=(12,12))

# Define the number of million reads to trim
trim_reads = 50

# Initialize variables
read_max = 0
n = 0
samples = []
a_list = []
b_list = []
max_reads = []
slope_list = []

# Define the logarithmic function used for fitting
def logFunc(x, a, b):
    return a + b*np.log(x)

# Loop through input histograms
for h in input_histograms:
    # Load the data from the current input histogram
    df = pd.read_csv(h, sep="\t")
    # Rename the columns to make them easier to work with
    df.columns = ["counts", "percent", "c", "d", "e", "f", "g", "h", "rand_cnt", "j"]
    # Drop unused columns
    df = df[["counts", "percent"]]
    # Calculate the percentage of known k-mers and convert the counts to millions of reads
    df["percent"] = (100 - df["percent"])
    df["counts"] = df["counts"] / 1000000
    # Plot the original data
    ax = sns.lineplot(x="counts", y="percent", data=df, err_style='band', color=colours[n], label=h.split("/")[-1].split("_kmer")[0], linewidth=0.5, alpha=0.8)
    # Keep track of the maximum read count
    read_max = max(read_max, df["counts"].max())
    # If there are enough data points, fit a logarithmic function to the tail of the data
    if len(df["counts"]) > trim_reads+1:
        # Fit the function to the tail of the data
        popt, pcov = curve_fit(logFunc, df["counts"][trim_reads:], df["percent"][trim_reads:])
        # Plot the fitted function
        ax = sns.lineplot(x=np.arange(0, trim_reads, 0.25), y=logFunc(np.arange(0, trim_reads, 0.25), *popt), color=colours[n], err_style='band', label=h.split("/")[-1].split("_kmer")[0] + "_fit", linewidth=0.5, alpha=0.5)
        # Store the fitting parameters and other statistics
        samples.append(h.split("/")[-1].split("_kmer")[0])
        a_list.append(popt[0])
        b_list.append(popt[1])
        max_reads.append(df["counts"].max())
        reads_to_1 = ((popt[0]) / (popt[1]))
        slope_list.append(popt[1] / df["counts"].max())
    # Add a vertical line to indicate the maximum read count
    plt.axvline(df["counts"].max(), 0, 1, linestyle="--", color=colours[n], alpha=0.4)
    # Increment the sample counter
    n = n+1
    
# Set the limits of the x and y axes
ax.set(ylim=(0, 110))
ax.set(xlim=(0, read_max*1.5))

# Set axis limits and labels, add legend, and save the figure
ax.set(ylim=(0, 110))
ax.set(xlim=(0, read_max*1.5))
ax.set_xlabel("Read count (Million reads)")
ax.set_ylabel("Known k-mers (%)")
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax.figure.savefig(output_kmer_png_fitted, format="png", bbox_inches = "tight",transparent=True)
ax.figure.savefig(output_kmer_svg_fitted, format="svg", bbox_inches = "tight",transparent=True)

# display the plot
plt.show()

In [ ]:
# create a new empty pandas DataFrame
stats_df=pd.DataFrame()

# add columns to the DataFrame using lists
stats_df["Sample"]=samples
stats_df["a"]=a_list
stats_df["b"]=b_list
stats_df["slope"]=slope_list
stats_df["M_reads"]=max_reads
stats_df["b_-_Mreads"]=stats_df["b"]-stats_df["M_reads"]

# sort the DataFrame by "Sample" column, apply background color gradient to it, and render it as an HTML table
stats_df_out=stats_df.sort_values(by="Sample").style.background_gradient(cmap="RdYlGn").render()

# write the HTML table to a file
with open(output_kmer_fit_html,"w") as fp:
    fp.write(stats_df_out)

# sort the DataFrame by "Sample" column and apply background color gradient to it
stats_df.sort_values(by="Sample").style.background_gradient(cmap="RdYlGn")

## PreQC and PostQC FASTQC statistics

In [ ]:
# load the pre- and post-illumina sequencing quality control data into pandas dataframes
preqc = pd.read_csv(illumina_preqc, sep="\t")
postqc = pd.read_csv(illumina_postqc, sep="\t")

# create a new dataframe with columns for Filename, average sequence length, and total sequences from the preqc dataframe
read_stats_df1 = preqc[["Filename","avg_sequence_length", "Total Sequences"]]

# calculate the total number of megabases for each file and add this as a new column to read_stats_df1
read_stats_df1["Mbp"] = read_stats_df1["avg_sequence_length"] * read_stats_df1["Total Sequences"] / 1000000

# remove the ".fastq" extension from the Filename column
read_stats_df1["Filename"] = read_stats_df1["Filename"].str.split(".fastq").str[0]

# rename the columns of read_stats_df1
read_stats_df1.columns = ["file_pre","length_pre", "number_pre", "Mbp_pre"]

# set the Filename column as the index of read_stats_df1
read_stats_df1 = read_stats_df1.set_index('file_pre')

# remove any whitespace from the index values of read_stats_df1
read_stats_df1.index = read_stats_df1.index.map("".join)

# create a new dataframe with columns for Filename, average sequence length, and total sequences from the postqc dataframe
read_stats_df2 = postqc[["Filename","avg_sequence_length", "Total Sequences"]]

# calculate the total number of megabases for each file and add this as a new column to read_stats_df2
read_stats_df2["Mbp"] = read_stats_df2["avg_sequence_length"] * read_stats_df2["Total Sequences"] / 1000000

# replace certain portions of the Filename column with standard suffixes (e.g. _forward -> _R1)
read_stats_df2["Filename"] = read_stats_df2["Filename"].str.replace("_forward", "_R1")
read_stats_df2["Filename"] = read_stats_df2["Filename"].str.replace("_reverse", "_R2")
read_stats_df2["Filename"] = read_stats_df2["Filename"].str.replace("_unpaired", "_U")
read_stats_df2["Filename"] = read_stats_df2["Filename"].str.split("_paired").str[0]
read_stats_df2["Filename"] = read_stats_df2["Filename"].str.split("_clean").str[0]

# create a new column in read_stats_df2 for the sample name, based on the first part of the Filename (before the final underscore)
read_stats_df2["Sample"] = read_stats_df2["Filename"].str.rsplit("_",1).str[0]

# set the Filename column as the index of read_stats_df2
read_stats_df2 = read_stats_df2.set_index('Filename')

# rename the columns of read_stats_df2
read_stats_df2.columns=["length_post", "number_post", "Mbp_post", "sample"]

# merge read_stats_df1 and read_stats_df2 on their indices (i.e. Filename), keeping all rows from both dataframes
read_stats_df3 = read_stats_df1.merge(read_stats_df2, left_index=True, right_index=True, how="outer")

# replace any NaN (missing) values in read_stats_df3 with 0
read_stats_df3 = read_stats_df3.fillna(0)

# return the final merged dataframe
read_stats_df3


In [ ]:
# Pre quality control fastQC statistics
preqc

In [ ]:
# Post quality control fastQC statistics
postqc

In [ ]:
# Group read statistics by sample
read_stats_df = read_stats_df3.groupby(["sample"]).sum()

# Calculate percentage of reads and Mbp kept
read_stats_df["%number_kept"] = (read_stats_df["number_post"] / read_stats_df["number_pre"] * 100).round(2)
read_stats_df["%Mbp_kept"] = (read_stats_df["Mbp_post"] / read_stats_df["Mbp_pre"] * 100).round(2)

# Calculate percentage of reads and Mbp removed
read_stats_df["%Mbp_removed"] = 100 - read_stats_df["%Mbp_kept"]
read_stats_df["%number_removed"] = 100 - read_stats_df["%number_kept"]

# sort the DataFrame by "Sample" column, apply background color gradient to it, and render it as an HTML table
read_stats_out=read_stats_df.drop(["length_pre", "length_post"], axis=1).style.background_gradient(cmap="RdYlGn").render()

# write the HTML table to a file
with open(output_qc_summary_html,"w") as fp:
    fp.write(stats_df_out)
    
# Drop unnecessary columns and apply a gradient color scheme to the resulting DataFrame
read_stats_df.drop(["length_pre", "length_post"], axis=1).style.background_gradient(cmap="RdYlGn")

### Percentage of kept/removed reads

In [ ]:
# set figure size and font scale for seaborn
plt.figure(figsize=(12,12))

# Define colors for the plot
colors = sns.color_palette("Paired", n_colors=2)

# Calculate the figure width based on the number of samples
fig_width = len(SAMPLES) * .4

# Create a LinearSegmentedColormap object using the colors defined above
cmap1 = LinearSegmentedColormap.from_list("my_colormap", colors)

# Create a bar plot of the percentage of reads kept and removed for each sample
ax = read_stats_df[["%number_kept","%number_removed"]].plot(kind='bar', stacked=True, colormap=cmap1, figsize=(fig_width,12),width=0.8)

# Add a legend to the plot
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# Add axis labels and a title to the plot
plt.xlabel('Sample')
plt.ylabel('Percentage of reads') 
plt.title('Percentage of kept reads')

# Set the limits of the y axis
ax.set(ylim=(0, 100))

# Save the plot as a PNG and SVG file
ax.figure.savefig(output_percentage_kept_reads_png, format="png", bbox_inches = "tight",transparent=True)
ax.figure.savefig(output_percentage_kept_reads_svg, format="svg", bbox_inches = "tight",transparent=True)

# Show the plot
plt.show()


### Percentage of kept/removed Mbp

In [ ]:
# set figure size and font scale for seaborn
plt.figure(figsize=(12,12))


# set color palette for the bar chart
colors = sns.color_palette("Paired", n_colors=2)

# set the width of the figure based on number of samples
fig_width=len(SAMPLES) * .4

# create a colormap from the color palette
cmap1 = LinearSegmentedColormap.from_list("my_colormap", colors)

# create a stacked bar chart of percentage of Mbp kept and removed
ax=read_stats_df[["%Mbp_kept","%Mbp_removed"]].plot(kind='bar', stacked=True,colormap=cmap1, figsize=(fig_width,12),width=0.8)

# add a legend to the chart
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# add x and y axis labels to the chart
plt.xlabel('Sample')
plt.ylabel('Percentage of Mbp') 

# add a title to the chart
plt.title('Percentage of kept Mpb', pad=20)

# Set the limits of the y axis
ax.set(ylim=(0, 100))

# save the chart as a PNG and SVG file
ax.figure.savefig(output_percentage_kept_Mbp_png, format="png", bbox_inches = "tight",transparent=True)
ax.figure.savefig(output_percentage_kept_Mbp_svg, format="svg", bbox_inches = "tight",transparent=True)

# show the chart
plt.show()


## Removed reads per QC step

In [ ]:
# Initialize empty lists and variables
sample_l=[]
read_types=[]
raw=[]
trimmed=[]
duk=[]
euk=[]
norm=[]
reads=[]

# Loop through each sample in read_stats_df and populate the lists with relevant information
for sample in (read_stats_df.index.to_list()):  
    # Add sample name and read type to lists
    sample_l.extend([sample]*3)
    read_types.extend(["R1", "R2", "U"])
    # Read in raw read counts for each read type and add to raw list
    raw.append(int(open(raw_dir + "/" + sample + forward_tag + "_read_count.txt" , 'r').readline().strip()))
    raw.append(int(open(raw_dir + "/" +  sample + reverse_tag + "_read_count.txt" , 'r').readline().strip()))
    raw.append(int(0))
    # Read in trimmed read counts for each read type and add to trimmed list
    trimmed.append(int(open(clean_dir + "/" + sample + "_forward_paired_read_count.txt" , 'r').readline().strip()))
    trimmed.append(int(open(clean_dir + "/" + sample + "_reverse_paired_read_count.txt" , 'r').readline().strip()))
    trimmed.append(int(open(clean_dir + "/" + sample + "_merged_unpaired.tot_read_count.txt" , 'r').readline().strip()))
    # Read in non-eukaryotic read counts for each read type and add to euk list
    euk.append(int(open(clean_dir + "/" + sample + "_forward_paired_noEuk.tot_read_count.txt" , 'r').readline().strip()))
    euk.append(int(open(clean_dir + "/" + sample + "_reverse_paired_noEuk.tot_read_count.txt" , 'r').readline().strip()))
    euk.append(int(open(clean_dir + "/" + sample + "_unpaired_noEuk.tot_read_count.txt" , 'r').readline().strip()))
    # Read in BBDuk-cleaned read counts for each read type and add to duk list
    duk.append(int(open(clean_dir + "/" + sample + "_forward_paired_clean.tot_read_count.txt" , 'r').readline().strip()))
    duk.append(int(open(clean_dir + "/" + sample + "_reverse_paired_clean.tot_read_count.txt" , 'r').readline().strip()))
    duk.append(int(open(clean_dir + "/" + sample + "_unpaired_clean.tot_read_count.txt" , 'r').readline().strip()))
    # Read in normalized read counts for each read type and add to norm list
    norm.append(int(open(clean_dir + "/" + sample + "_forward_paired_norm.tot_read_count.txt" , 'r').readline().strip()))
    norm.append(int(open(clean_dir + "/" + sample + "_reverse_paired_norm.tot_read_count.txt" , 'r').readline().strip()))
    norm.append(int(open(clean_dir + "/" + sample + "_unpaired_norm.tot_read_count.txt" , 'r').readline().strip()))

# Create an empty pandas DataFrame
df_counts = pd.DataFrame()

# Add columns to the DataFrame
df_counts["sample"] = sample_l
df_counts["type"] = read_types
df_counts["raw"] = raw
df_counts["trimmomatic"] = trimmed
df_counts["kraken"] = euk
df_counts["bbduk"] = duk
df_counts["norm"] = norm

# Calculate values for new columns and add them to the DataFrame
df_counts["raw_s"] = df_counts["raw"] / 1000000
df_counts["low_QC_reads_s"] = (df_counts["raw"] - df_counts["trimmomatic"]) / 1000000
df_counts["eukaryotic_reads_s"] = (df_counts["trimmomatic"] - df_counts["kraken"]) / 1000000
df_counts["bbduk_phix174_reads_s"] = (df_counts["kraken"] - df_counts["bbduk"]) / 1000000
df_counts["duplicate_reads_s"] = (df_counts["bbduk"] - df_counts["norm"]) / 1000000
df_counts["assembly_reads_s"] = df_counts["norm"] / 1000000

df_counts["low_QC_reads_p"] = df_counts["low_QC_reads_s"] / df_counts["raw_s"]
df_counts["eukaryotic_reads_p"] = df_counts["eukaryotic_reads_s"] / df_counts["raw_s"]
df_counts["bbduk_phix174_reads_p"] = df_counts["bbduk_phix174_reads_s"] / df_counts["raw_s"]
df_counts["duplicate_reads_p"] = df_counts["duplicate_reads_s"] / df_counts["raw_s"]
df_counts["assembly_reads_p"] = df_counts["assembly_reads_s"] / df_counts["raw_s"]

# Create a new column by concatenating two existing columns
df_counts["sample_long"] = df_counts["sample"] + "_" + df_counts["type"]

# Convert the DataFrame to an HTML table and write it to a file
df_counts.to_html(output_step_qc_reads_html)

# Display the final DataFrame
df_counts



### Number of kept/removed reads per QC step

In [ ]:
# set figure size and font scale for seaborn
plt.figure(figsize=(12,12))

# Set color palette and figure width
colors = sns.color_palette("Paired", n_colors=5)
fig_width = len(SAMPLES) * 0.4

# Define legend text and create colormap
legend_text = ["Used for assembly", "Duplicate (removed)", "phiX174 (user contaminants)", "Eukaryotic contamination", "Low quality"]
cmap1 = LinearSegmentedColormap.from_list("my_colormap", colors)

# Filter the dataframe to only include R1 reads and plot a stacked bar chart
df_counts_paired = df_counts[df_counts["type"] == "R1"]
ax = df_counts_paired.plot(x="sample", y=["assembly_reads_s", "duplicate_reads_s", "bbduk_phix174_reads_s", "eukaryotic_reads_s", "low_QC_reads_s"], kind="bar", stacked=True, colormap=cmap1, figsize=(fig_width, 12),width=0.8)

# Add legend, axis labels, and save the figure
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.xlabel("Sample")
plt.ylabel("Million reads") 
plt.savefig(output_steps_qc_reads_png, format="png", bbox_inches = "tight",transparent=True)
plt.savefig(output_steps_qc_reads_svg, format="svg", bbox_inches = "tight",transparent=True)

# Show the plot
plt.show()


### Percentage of kept/removed reads per QC step

In [ ]:
# set figure size and font scale for seaborn
plt.figure(figsize=(12,12))

# Set colors for the stacked bar chart
colors = sns.color_palette("Paired", n_colors=5)

# Set figure width based on number of samples
fig_width = len(SAMPLES) * .4

# Create a colormap
cmap1 = LinearSegmentedColormap.from_list("my_colormap", colors)

# Select only R1 data from df_counts
df_counts_paired = df_counts[df_counts["type"] == "R1"]

# Plot stacked bar chart with proportion of raw reads
ax = df_counts_paired.plot(
    x="sample",
    y=["assembly_reads_p", "duplicate_reads_p", "bbduk_phix174_reads_p", "eukaryotic_reads_p", "low_QC_reads_p"],
    kind="bar",
    stacked=True,
    colormap=cmap1,
    figsize=(fig_width, 12),
    width=0.8
)

# Add legend
legend_text = ["Used for assembly", "Duplicate (removed)", "phiX174 (user contaminants)", "Eukaryotic contamination", "Low quality"]
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# Set x and y labels
plt.xlabel("Sample")
plt.ylabel("Proportion of raw reads")

# Set the limits of the y axis
ax.set(ylim=(0, 1))

# Save the figure as PNG and SVG files
plt.savefig(output_steps_qc_percentage_png, format="png", bbox_inches = "tight",transparent=True)
plt.savefig(output_steps_qc_percentage_svg, format="svg", bbox_inches = "tight",transparent=True)

# Display the plot
plt.show()


In [ ]:
df_counts_paired

## Super Deduper PCR duplication statistics

In [ ]:
# set figure size and font scale for seaborn
plt.figure(figsize=(12,12))

# initialize variables
maxim=0
n=0
percentages=[]

# loop through each sample
for sample in SAMPLES:   
    # open the file and load the data
    f = open(qc_dir + "/" +sample + "_stats_pcr_duplicates.log")
    data=json.load(f)
    
    # extract the relevant data and calculate percentage of duplicates
    df=pd.DataFrame(data[0]["Fragment"]["duplicate_saturation"], columns=["reads","dup"] )/1000000
    percentages.append(df.iloc[-1]["dup"]*100/df.iloc[-1]["reads"])
    
    # find the maximum value of "reads"
    max_temp=df["reads"].max()
    if max_temp>maxim: maxim=max_temp;
    
    # plot the data as a line plot using seaborn
    ax = sns.lineplot(x="reads", y="dup", data=df,err_style='band',color=colours[n], label=sample, linewidth=1, alpha=0.8)
    n=n+1

# set the x and y limits of the plot
ax.set(ylim=(0,maxim))
ax.set(xlim=(0, maxim))

# set the x and y labels of the plot
ax.set_xlabel("Million reads")
ax.set_ylabel("PCR duplicates (Million reads)")

# add a legend to the plot
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# save the plot as png and svg files
ax.figure.savefig(output_supperdedupper_png, format="png", bbox_inches = "tight",transparent=True)
ax.figure.savefig(output_supperdedupper_svg, format="svg", bbox_inches = "tight",transparent=True)

# create a dataframe to store the percentage of duplicates for each sample
pcr_dup_df=pd.DataFrame()
pcr_dup_df["sample"]=SAMPLES
pcr_dup_df["pcr_percent_duplicates"]=percentages

# generate a styled HTML table of the dataframe and save it to a file
pcr_dup_df_out=pcr_dup_df.style.background_gradient(cmap="RdYlGn_r", vmin=0, vmax=100).render()
with open(output_supperdedupper_html,"w") as fp:
    fp.write(pcr_dup_df_out)


In [ ]:
# display the styled dataframe in the notebook
pcr_dup_df.style.background_gradient(cmap="RdYlGn_r", vmin=0, vmax=100)